# TP n°3: Suivi Multi-Objet
**Membres:**
- Guillaume Girouard 1954899
- Pierre-Emmanuel Rebours 2165286

On importe les librairies nécessaires

In [ ]:
!pip install ocsort

In [ ]:
from ocsort.ocsort import OCSort as OCSort

In [ ]:
#!git clone https://github.com/noahcao/OC_SORT.git
!git clone https://github.com/JonathonLuiten/TrackEval.git

In [ ]:
!wget https://omnomnom.vision.rwth-aachen.de/data/TrackEval/data.zip
!unzip data.zip -d TrackEval

In [ ]:
!pip install natsort

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import numpy as np

#from OC_SORT.trackers.ocsort_tracker.ocsort import OCSort
from dataloader import dataloader

In [ ]:
import tensorflow as tf

In [ ]:
pip install filterpy

### Récupération des résultats pour MOT17

Les commandes suivantes permettent d'installer les données du dataset MOT17 mais aussi de préparer l'utilisation de TrackEval.

In [ ]:
!wget https://motchallenge.net/data/MOT17.zip

--2023-04-08 19:44:06--  https://motchallenge.net/data/MOT17.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5860214001 (5.5G) [application/zip]
Saving to: ‘MOT17.zip’

MOT17.zip           100%[===================>]   5.46G  26.5MB/s    in 4m 39s  

2023-04-08 19:48:46 (20.1 MB/s) - ‘MOT17.zip’ saved [5860214001/5860214001]



In [ ]:
!unzip MOT17.zip

In [ ]:
!mkdir ./TrackEval/data/trackers/mot_challenge/MOT17_tp-train

In [ ]:
!mkdir ./TrackEval/data/trackers/mot_challenge/MOT17_tp-train/OC-SORT

In [ ]:
!mkdir ./TrackEval/data/trackers/mot_challenge/MOT17_tp-train/OC-SORT/data

In [ ]:
!mkdir ./TrackEval/data/gt/mot_challenge/MOT17_tp-train

In [ ]:
!cp -R ./TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-11-DPM ./TrackEval/data/gt/mot_challenge/MOT17_tp-train

In [ ]:
!echo -e 'name\nMOT17-02-DPM\nMOT17-04-DPM\nMOT17-05-DPM\nMOT17-09-DPM\nMOT17-10-DPM\nMOT17-11-DPM\nMOT17-13-DPM' > './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt'

In [ ]:
!cp './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt' './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-all.txt'

In [ ]:
!cp './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt' './TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-test.txt'

In [ ]:
nb_frame=dict({"MOT17-02-DPM":600,"MOT17-04-DPM":1050,"MOT17-05-DPM":837,
               "MOT17-09-DPM":525,"MOT17-10-DPM":654,"MOT17-11-DPM":900,
               "MOT17-13-DPM":750})

In [ ]:
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-02-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-04-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'

In [ ]:
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-05-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-09-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-10-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-11-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'
!cp -R './TrackEval/data/gt/mot_challenge/MOT17-train/MOT17-13-DPM' './TrackEval/data/gt/mot_challenge/MOT17_tp-train/'

On importe le modèle utilisé pour la detection: Yolov5x

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-8 Python-3.9.16 torch-2.0.0+cu118 CPU



[Errno 2] No such file or directory: '/usr/local/lib/python3.9/dist-packages/scipy-1.10.1.dist-info/METADATA'


100%|██████████| 166M/166M [00:01<00:00, 147MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
def get_result(dl,nb_frame):
  """
    fonction pour appliquer notre méthode sur un dataframe et enregistrer les résultats dans un fichier .txt
    Args:
        dl: dataloader important un dataset
        nb_frame: nombre de frame du dataset récupérer par le dataloader
  """
  #Il faut réinitialiser le traceur à chaque nouveau dataset:
  tracker = OCSortK()
  tracker.det_thresh=0.5
  for i in range(nb_frame):
    image=dl.load_input(i)
    img_size=np.shape(image)

    #Détection
    results = model(image)
    res=results.pandas().xyxy[0][['xmin','ymin','xmax','ymax','confidence','class']]
    cup_results=res[res['class']==0]
    np_res=cup_results.to_numpy()
    tf_res=tf.constant(np_res)
    #suivi
    online_targets = tracker.update(tf_res,_)
    dl.load_result(i+1,online_targets,benchmark_type='MOT')
  dl.f.close()


Récupération des résultats pour tous les dataframe de MOT17_tp

In [ ]:
f=open('./TrackEval/data/gt/mot_challenge/seqmaps/MOT17_tp-train.txt')
for i,row in enumerate(f):
  if i>=1:
    row=row[:-1]
    dl=dataloader("./MOT17/train/"+row+"/img1/","./TrackEval/data/trackers/mot_challenge/MOT17_tp-train/OC-SORT/data/"+row+".txt")
    get_result(dl,nb_frame[row])
    print("Résultat du dataset "+row+" disponible")
f.close()

Résultat du dataset MOT17-13-DPM disponible


Calcul des performances

In [ ]:
!python TrackEval/scripts/run_mot_challenge.py --BENCHMARK MOT17_tp --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL OC-SORT --METRICS HOTA --USE_PARALLEL False --NUM_PARALLEL_CORES 1


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 1                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /content/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /content/TrackEval/data/gt/mot_challenge/
TRACKERS_FOLDER      : /content/TrackEval/data/trackers/mot_challenge/
OUTPUT_FOL

In [ ]:
!python TrackEval/scripts/run_mot_challenge.py --BENCHMARK MOT17_tp --SPLIT_TO_EVAL train --TRACKERS_TO_EVAL OC-SORT --METRICS HOTA --USE_PARALLEL False --NUM_PARALLEL_CORES 1

### Récupération des résultats pour la vidéo du tp

Importation du modèle Yolo5vx

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True)

/usr/local/lib/python3.9/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-4-12 Python-3.9.16 torch-2.0.0+cu118 CPU



[Errno 2] No such file or directory: '/usr/local/lib/python3.9/dist-packages/scipy-1.10.1.dist-info/METADATA'


100%|██████████| 166M/166M [00:01<00:00, 88.2MB/s]

Fusing layers... 
YOLOv5x summary: 444 layers, 86705005 parameters, 0 gradients
Adding AutoShape... 


Récupération des résultats

In [ ]:
dl=dataloader("./frames/","./results.txt")
nb_frame=len(dl.input_data)
tracker = OCSort()
tracker.det_thresh=0.5
for i in range(nb_frame):
  image=dl.load_input(i)
  img_size=np.shape(image)
  
  #détection
  results = model(image)
  res=results.pandas().xyxy[0][['xmin','ymin','xmax','ymax','confidence','class']]
  cup_results=res[res['class']==41]
  np_res=cup_results.to_numpy()
  tf_res=tf.constant(np_res)
  #suivi
  online_targets = tracker.update(tf_res,_)#,img_size,img_size)
  dl.load_result(i,online_targets)
dl.f.close()